In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/cs/cs171/project

/content/drive/MyDrive/cs/cs171/project


In [ ]:
!pip install billboard.py

In [ ]:
!pip -q install pandas pyarrow

In [ ]:
import billboard as b
import pandas as pd, re, json
import os
import numpy as np
from collections import Counter

In [ ]:
lyrics_data = 'billboard_24years_lyrics_spotify.csv'
df = pd.read_csv(lyrics_data)
df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,0.000000,0.2510,0.278,136.859,audio_features,3y4LxiYMgDl4RethdzpmNe,https://api.spotify.com/v1/tracks/3y4LxiYMgDl4...,https://api.spotify.com/v1/audio-analysis/3y4L...,250547.0,4.0
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,0.000005,0.2950,0.961,115.996,audio_features,0n2SEXB2qoRQg171q7XqeW,https://api.spotify.com/v1/tracks/0n2SEXB2qoRQ...,https://api.spotify.com/v1/audio-analysis/0n2S...,294987.0,4.0
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.590,...,0.807000,0.2990,0.724,115.983,audio_features,5IALWUYK0zDSEmZgb4ICvc,https://api.spotify.com/v1/tracks/5IALWUYK0zDS...,https://api.spotify.com/v1/audio-analysis/5IAL...,244924.0,4.0
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,0.002010,0.0348,0.680,97.911,audio_features,3XKIUb7HzIF1Vu9usunMzc,https://api.spotify.com/v1/tracks/3XKIUb7HzIF1...,https://api.spotify.com/v1/audio-analysis/3XKI...,261973.0,4.0
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ranking           3397 non-null   int64  
 1   song              3397 non-null   object 
 2   band_singer       3397 non-null   object 
 3   songurl           3397 non-null   object 
 4   titletext         3397 non-null   object 
 5   url               3397 non-null   object 
 6   year              3397 non-null   int64  
 7   lyrics            3397 non-null   object 
 8   uri               3397 non-null   object 
 9   danceability      486 non-null    float64
 10  energy            486 non-null    float64
 11  key               486 non-null    float64
 12  loudness          486 non-null    float64
 13  mode              486 non-null    float64
 14  speechiness       486 non-null    float64
 15  acousticness      486 non-null    float64
 16  instrumentalness  486 non-null    float64


In [ ]:
df['lyrics']

,lyrics
0,I can feel the magic floating in the air\nBein...
1,"Man, it's a hot one\nLike seven inches from th..."
2,"Man, it's a hot one\nLike seven inches from th..."
3,"Ladies and gents, turn up your sound systems\n..."
4,Turn up this sound system\nTo the sound of Car...
...,...
3392,It was right after senior year\nJust before th...
3393,When I need motivation\nMy one solution is my ...
3394,When I need motivation\nMy one solution is my ...
3395,My face above the water\nMy feet can't touch t...


In [ ]:
temp = df.loc[2, 'lyrics']
temp

'Man, it\'s a hot one\nLike seven inches from the midday sun\nWell, I hear you whisper and the words melt everyone\nBut you stay so cool\nMy muñequita, my Spanish Harlem Mona Lisa\nYou\'re my reason for reason, the step in my groove, yeah\n\nAnd if you say, "This life ain\'t good enough"\nI would give my world to lift you up\nI could change my life to better suit your mood\n\'Cause you\'re so smooth\n\nAnd it\'s just like the ocean under the moon\nWell, it\'s the same as the emotion that I get from you\nYou got the kind of lovin\' that can be so smooth\nGimme your heart, make it real, or else forget about it\n\nWell, I\'ll tell you one thing\nIf you would leave it\'d be a crying shame\nIn every breath and every word\nI hear your name callin\' me out\nOut from the barrio, you hear my rhythm on the radio\nYou feel the turning of the world so soft and slow\nTurnin\' you round and round\nYou might also like\nAnd if you say, "This life ain\'t good enough"\nI would give my world to lift you 

#Find abbrviate word and make function

In [ ]:
df = pd.read_csv('billboard_24years_lyrics_spotify.csv')

norm = lambda s: str(s).replace("’","'").replace("`","'")
word_checker = re.compile(r"\b[A-Za-z]+'[A-Za-z]+\b|\b'[A-Za-z]+\b|\b[A-Za-z]+'\b")

counts = Counter()
for t in df["lyrics"].dropna().astype(str):
    t = norm(t)
    for w in word_checker.findall(t):
        counts[w.lower()] += 1

cont_word_df = pd.DataFrame(counts.most_common(), columns=["token","count"])
cont_word_df.head(10)

,token,count
0,i'm,19920
1,don't,12287
2,it's,8200
3,ain't,6075
4,you're,5305
5,can't,5090
6,i'll,3439
7,that's,3185
8,let's,1934
9,i've,1914


In [ ]:
contraction_df = cont_word_df.copy()
contraction_df['replace'] = ''
contraction_df.to_csv('contraction_word.csv', index=False)
contraction_df.head(20)

,token,count,replace
0,i'm,19920,
1,don't,12287,
2,it's,8200,
3,ain't,6075,
4,you're,5305,
5,can't,5090,
6,i'll,3439,
7,that's,3185,
8,let's,1934,
9,i've,1914,


In [ ]:
norm = lambda s: str(s).replace("’","'").replace("`","'")
pat = re.compile(r"\b[A-Za-z]+'[A-Za-z]+\b|\b[A-Za-z]+'\b")

tokens = []
for t in df["lyrics"].dropna().astype(str):
    t = norm(t)
    tokens.extend(pat.findall(t))

apos_word = pd.DataFrame(Counter(w.lower() for w in tokens).most_common(), columns=["token","count"])
apos_word

,token,count
0,i'm,19920
1,don't,12287
2,it's,8200
3,ain't,6075
4,you're,5305
...,...,...
1003,olvida'o,1
1004,breeze'll,1
1005,religion's,1
1006,j'veux,1


#Function to tokenize sentence -> word

In [ ]:
TOKEN_RE = re.compile(r"[A-Za-z]+")
